# Day 5: Best Locations

### Problem:

Hi, FlORence ici 🇫🇷

I'm a manager of AmazOR that wants to find the best possible locations for our warehouses while deciding how to allocate our client demands to these warehouses.

Our warehouses can only handle a limited amount of demand but we need to ensure that all client demands are met.

While opening a new facility, we have some fixed costs, and we also take into account the cost of allocating the demand of each client to each warehouse.

Of course, we want to minimize our costs here...

Can you help me solve this problem?

In instance.txt we can find an instance of the problem.

### Solution:

The aim of this problem is to minimize the cost of opening the warehouses and minimize the cost of allocate the demand of each client to each warehouse. Hence, the variables of our model will be the following:
- $x_{ij}$, the units of client $i$ stored in warehouse $j$.
- $y_{i}$, binary variable, $1$ if the warehouse $i$ is opened.

Therefore, the objective function for our problem will be,
$$
\text{minimize} \sum_{i\in\text{n}_{\text{W}}} C_{i}^{\text{open}}y_{i} + \sum_{i\in\text{n}_{\text{W}}}\sum_{j\in\text{n}_{\text{C}}} C_{ij}^{\text{client}}x_{ij}
$$
where $C_{i}^{\text{open}}$ correspond the cost of opening warehouse $i$ and $C_{ij}^{\text{client}}$ correspond the cost of handling the demand of client $j$ in warehouse $i$.

The constrains of maximum capacity of each warehouse and each client has satisfied his demand has to be implemented. 

Finally, the complete mathematical formulation for this problem is the following,

$$
\begin{align*}
\text{maximize} & \sum_{i\in\text{n}_{\text{W}}} C_{i}^{\text{open}}y_{i} + \sum_{i\in\text{n}_{\text{W}}}\sum_{j\in\text{n}_{\text{C}}} C_{ij}^{\text{client}}x_{ij}\\
\text{subject to} & \sum_{i\in\text{n}_{\text{W}}} x_{ij} = D_{j} \quad \forall j\in n_{C}\\
& \sum_{j\in\text{n}_{\text{C}}} x_{ij} \leq 5000 y_{i} \quad \forall i \in n_{W}\\
& x_{ij} \geq 0 \quad \forall i,j \\
& y_{i} \in\lbrace 0,1\rbrace \quad \forall i
\end{align*}
$$

where $D_{j}$ correspond the demand of client $j$.

Once the problem is formulated, the next step is to solve it using Pyomo for the cost matrix $C^{\text{open}}$, demand vector $D$ and the store cost $C^{\text{client}}$ are available in `instance.txt`.

In [1]:
# Set the initial variables of the problem
N_WAREHOUSES = 16
N_CLIENTS = 50
MAX_DEMAND = 5000
COST_OPEN_LOCATION = [7500] * N_WAREHOUSES
COST_OPEN_LOCATION[10] = 0

In [2]:
import numpy as np

# Read the demand matrix, and cost of store
file_path = "instance.txt"

with open(file_path, 'r') as file:
    lines = file.readlines()

demand = []
cost_stored = np.zeros((N_WAREHOUSES, N_CLIENTS), dtype=float)
# Extract the requirement matrix per room
for c in range(N_CLIENTS):
    start_idx = 29+c*4          # Start index for the current task row

    demand.append(int(lines[start_idx]))
    #for w in range(N_WAREHOUSES):
    row = list(map(float, " ".join(lines[start_idx+1:start_idx+4]).split()))
    cost_stored[:,c] = np.asarray(row) #/ demand[c]
    

# Check the matrix
#print(cost_stored)

Implementation in Pyomo

In [3]:
from pyomo.environ import *

# Initialize the model
model = ConcreteModel()

# Sets
model.C = RangeSet(N_CLIENTS)
model.W = RangeSet(N_WAREHOUSES)

### Variables
model.x = Var(model.W, model.C, domain=NonNegativeReals)
model.y = Var(model.W, domain=Binary)

### Constraint
# Client demand fulfilled
def client_demand(model, c):
    return sum(model.x[w, c] for w in model.W) == demand[c-1]
model.ClientDemand = Constraint(model.C, rule=client_demand)

# Warehouse capacity limit
def warehouse_capacity(model, w):
    return sum(model.x[w,c] for c in model.C) <= MAX_DEMAND * model.y[w]
model.WarehouseCapacity = Constraint(model.W, rule=warehouse_capacity)


# Objective
# Minimize the cost
def objective_rule(model):
    return sum(COST_OPEN_LOCATION[w-1]*model.y[w] for w in model.W) + sum(cost_stored[w-1,c-1]*model.x[w,c] for w in model.W for c in model.C)
model.Objective = Objective(rule=objective_rule, sense=minimize)


# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)

# Check and output results
if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
    warehouse_clients = {}
    for w in model.W:
        if model.y[w].value == 1:
            print(f"Warehouse {w} is open")
            clients = []
            for c in model.C:
                if model.x[w, c].value > 0:
                    clients.append(f"Client {c}, units {model.x[w,c].value}")
            warehouse_clients[f"Warehouse {w}"] = clients
else:
    print("No optimal solution found.")

Warehouse 1 is open
Warehouse 2 is open
Warehouse 3 is open
Warehouse 4 is open
Warehouse 5 is open
Warehouse 6 is open
Warehouse 7 is open
Warehouse 8 is open
Warehouse 9 is open
Warehouse 10 is open
Warehouse 11 is open
Warehouse 12 is open
Warehouse 13 is open
Warehouse 14 is open
Warehouse 15 is open
Warehouse 16 is open


In [4]:
for warehouse, clients in warehouse_clients.items():
    print(f"{warehouse}:")
    for client in clients:
        print(f"  - {client}")

Warehouse 1:
  - Client 3, units 672.0
  - Client 4, units 1337.0
  - Client 6, units 559.0
  - Client 14, units 143.0
  - Client 24, units 304.0
  - Client 26, units 337.0
  - Client 40, units 328.0
  - Client 49, units 1281.0
Warehouse 2:
  - Client 7, units 2088.0
  - Client 34, units 2912.0
Warehouse 3:
  - Client 34, units 5000.0
Warehouse 4:
  - Client 11, units 5000.0
Warehouse 5:
  - Client 12, units 904.0
  - Client 37, units 3671.0
  - Client 38, units 242.0
  - Client 49, units 183.0
Warehouse 6:
  - Client 34, units 5000.0
Warehouse 7:
  - Client 15, units 615.0
  - Client 20, units 195.0
  - Client 44, units 500.0
  - Client 48, units 49.0
Warehouse 8:
  - Client 1, units 146.0
  - Client 5, units 31.0
  - Client 9, units 33.0
  - Client 10, units 32.0
  - Client 16, units 564.0
  - Client 39, units 705.0
  - Client 43, units 275.0
  - Client 45, units 1609.0
  - Client 46, units 733.0
  - Client 47, units 222.0
Warehouse 9:
  - Client 7, units 282.0
  - Client 8, units 10